# Solve Any Data Analysis Problem

## Chapter 10 - Project 7 - Example solution - Part 1

Data from: https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads#yearly-file

Data dictionary: https://www.kaggle.com/datasets/hm-land-registry/uk-housing-prices-paid

- **Transaction unique identifier** A reference number which is generated automatically recording each published sale. The number is unique and will change each time a sale is recorded.
- **Price** Sale price stated on the transfer deed.
- **Date of Transfer** Date when the sale was completed, as stated on the transfer deed.
- **Property Type** D = Detached, S = Semi-Detached, T = Terraced, F = Flats/Maisonettes, O = Other

Note that:

    we only record the above categories to describe property type, we do not separately identify bungalows.
    end-of-terrace properties are included in the Terraced category above.
    ‘Other’ is only valid where the transaction relates to a property type that is not covered by existing values.
  
- **Old/New** Indicates the age of the property and applies to all price paid transactions, residential and non-residential.
Y = a newly built property, N = an established residential building
- **Duration** Relates to the tenure: F = Freehold, L= Leasehold etc.

Note that HM Land Registry does not record leases of 7 years or less in the Price Paid Dataset.
- **Town/City**
- **District**
- **County**
- **PPD Category Type** Indicates the type of Price Paid transaction.
A = Standard Price Paid entry, includes single residential property sold for full market value.
B = Additional Price Paid entry including transfers under a power of sale/repossessions, buy-to-lets (where they can be identified by a Mortgage) and transfers to non-private individuals. Note that category B does not separately identify the transaction types stated. HM Land Registry has been collecting information on Category A transactions from January 1995. Category B transactions were identified from October 2013.
- **Record Status** - monthly file only Indicates additions, changes and deletions to the records.(see guide below).
A = Addition
C = Change
D = Delete.

Column names are a combination of the above and address column details from https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads#using-or-publishing-our-price-paid-data

In [1]:
import pandas as pd

prices_2021 = pd.read_csv("./data/pp-2021.csv.gz", nrows=1000)
prices_2021.head()

,{D707E535-5720-0AD9-E053-6B04A8C067CC},260000,2021-08-06 00:00,SO45 2HT,T,N,F,17,Unnamed: 8,PERRYWOOD CLOSE,HOLBURY,SOUTHAMPTON,NEW FOREST,HAMPSHIRE,A,A.1
0,{D707E535-5721-0AD9-E053-6B04A8C067CC},375000,2021-09-01 00:00,SO23 7FR,S,N,F,1,NaN,BOXALL GARDENS,KINGS WORTHY,WINCHESTER,WINCHESTER,HAMPSHIRE,A,A
1,{D707E535-5723-0AD9-E053-6B04A8C067CC},132000,2021-06-28 00:00,SP11 6RL,F,N,L,2,NaN,SEDGE ROAD,NaN,ANDOVER,TEST VALLEY,HAMPSHIRE,A,A
2,{D707E535-5724-0AD9-E053-6B04A8C067CC},295000,2021-09-10 00:00,SP11 6TU,T,N,F,21,NaN,NAP CLOSE,NaN,ANDOVER,TEST VALLEY,HAMPSHIRE,A,A
3,{D707E535-5725-0AD9-E053-6B04A8C067CC},360000,2021-08-27 00:00,SO51 0AX,T,N,F,130,NaN,FREEMANTLE ROAD,NaN,ROMSEY,TEST VALLEY,HAMPSHIRE,A,A
4,{D707E535-5726-0AD9-E053-6B04A8C067CC},490500,2021-06-30 00:00,GU14 6JH,D,N,F,17,NaN,CLOSEWORTH ROAD,NaN,FARNBOROUGH,RUSHMOOR,HAMPSHIRE,A,A


Merge the three datasets

In [2]:
annual_dfs = []

for year in [2021, 2022, 2023]:
    print(f"Parsing {year}")
    df = pd.read_csv(
        f"./data/pp-{year}.csv.gz",
        names=["transaction_id", "sale_price", "sale_date", "postcode",
               "property_type", "old_new", "duration", "house_number_name",
               "second_addressable_object_name", "street", "locality",
               "town_city", "district", "county",
               "category_type", "record_status"],
        parse_dates=["sale_date"])
    annual_dfs.append(df)

price_paid = pd.concat(annual_dfs, axis=0, ignore_index=True)
print(price_paid.shape)
price_paid.head()

Parsing 2021
Parsing 2022
Parsing 2023
(2800288, 16)


,transaction_id,sale_price,sale_date,postcode,property_type,old_new,duration,house_number_name,second_addressable_object_name,street,locality,town_city,district,county,category_type,record_status
0,{D707E535-5720-0AD9-E053-6B04A8C067CC},260000,2021-08-06,SO45 2HT,T,N,F,17,Unnamed: 8,PERRYWOOD CLOSE,HOLBURY,SOUTHAMPTON,NEW FOREST,HAMPSHIRE,A,A.1
1,{D707E535-5721-0AD9-E053-6B04A8C067CC},375000,2021-09-01,SO23 7FR,S,N,F,1,NaN,BOXALL GARDENS,KINGS WORTHY,WINCHESTER,WINCHESTER,HAMPSHIRE,A,A
2,{D707E535-5723-0AD9-E053-6B04A8C067CC},132000,2021-06-28,SP11 6RL,F,N,L,2,NaN,SEDGE ROAD,NaN,ANDOVER,TEST VALLEY,HAMPSHIRE,A,A
3,{D707E535-5724-0AD9-E053-6B04A8C067CC},295000,2021-09-10,SP11 6TU,T,N,F,21,NaN,NAP CLOSE,NaN,ANDOVER,TEST VALLEY,HAMPSHIRE,A,A
4,{D707E535-5725-0AD9-E053-6B04A8C067CC},360000,2021-08-27,SO51 0AX,T,N,F,130,NaN,FREEMANTLE ROAD,NaN,ROMSEY,TEST VALLEY,HAMPSHIRE,A,A


In [3]:
price_paid["sale_date"].agg(["min", "max"])

min   2021-01-01
max   2023-12-31
Name: sale_date, dtype: datetime64[ns]

In [4]:
price_paid.to_csv("./data/price_paid.csv.gz", index=False)